In [16]:
import pandas as pd
import json
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

OSError: /opt/conda/lib/python3.7/site-packages/nvidia/cublas/lib/libcublas.so.11: symbol cublasLtHSHMatmulAlgoInit version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

In [2]:
queryData = pd.read_csv('gs://data_tql/spider/processed/spiderQueryData.csv')
tableData = pd.read_csv('gs://data_tql/spider/processed/Schemas/tablesSchemaSpider.csv')

display(queryData.head(1))
display(tableData.head(2))

,db_id,TQL,SQL,dataset,fileName,filePath,result
0,department_management,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56,train,department_management.sqlite,sqliteDB/department_management.sqlite,{'count(*)': {0: 5}}


,schema_id,table_name,table_name_original,primary_key,column_list,column_list_original,column_datatypes,foreign_keys
0,perpetrator,perpetrator,perpetrator,Perpetrator_ID,"['perpetrator id', 'people id', 'date', 'year'...","['Perpetrator_ID', 'People_ID', 'Date', 'Year'...","['number', 'number', 'text', 'number', 'text',...",[]
1,perpetrator,people,people,People_ID,"['people id', 'name', 'height', 'weight', 'hom...","['People_ID', 'Name', 'Height', 'Weight', 'Hom...","['number', 'text', 'number', 'number', 'text']","[['perpetrator', 'People_ID', 'people', 'Peopl..."


In [7]:
def create_schema_natural_language(row):

    schema_id = row['schema_id']
    table_name = row['table_name']
    primary_key = row['primary_key']
    column_list = eval(row['column_list_original'])
    datatype_list = eval(row['column_datatypes'])
    foreign_key = eval(row['foreign_keys'])

    column_list_with_datatype = []
    for column, datatype in zip(column_list, datatype_list):
        column_list_with_datatype.append(' has datatype '.join([column, datatype]))

    schema_natural_language = f"Given the Table {table_name} having columns as {', '.join(column_list_with_datatype)} which has {primary_key}"
    return schema_natural_language

In [8]:
tableData['schema_natural_language'] = tableData.apply(create_schema_natural_language, axis = 1)
tableData.head(3)

all_schemas = tableData['schema_id'].unique()
schema_table_query = {}
for schema in all_schemas:
    schema_details = ' and '.join(tableData[tableData['schema_id'] == schema]['schema_natural_language'].values)
    schema_table_query[schema] = schema_details

queryData['schema_natural_language'] = queryData['db_id'].map(schema_table_query)
queryData['final_TQL'] = queryData['TQL'] + ' ' + queryData['schema_natural_language']
queryData.head(2)

queryData['final_TQL'][0], queryData['SQL'][0]

('How many heads of the departments are older than 56 ? Given the Table department having columns as Department_ID has datatype number, Name has datatype text, Creation has datatype text, Ranking has datatype number, Budget_in_Billions has datatype number, Num_Employees has datatype number which has Department_ID and Given the Table head having columns as head_ID has datatype number, name has datatype text, born_state has datatype text, age has datatype number which has head_ID and Given the Table management having columns as department_ID has datatype number, head_ID has datatype number, temporary_acting has datatype text which has department_ID',
 'SELECT count(*) FROM head WHERE age  >  56')

In [10]:
# Load the pretrained T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

NameError: name 'T5Tokenizer' is not defined

In [ ]:
# Define a custom dataset for training
class SQLDataset(Dataset):
    def __init__(self, input_texts, target_queries, tokenizer, task_prefix):
        self.input_texts = input_texts
        self.target_queries = target_queries
        self.tokenizer = tokenizer
        self.task_prefix = task_prefix

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, index):
        
        input_text = self.task_prefix + self.input_texts[index]
        target_query = self.target_queries[index]

        input_encoding = self.tokenizer([input_text], return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        target_encoding = self.tokenizer([target_query], return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        
        return {
            'input_ids': input_encoding.input_ids.squeeze(0),
            'attention_mask': input_encoding.attention_mask.squeeze(0),
            'labels': target_encoding.input_ids.squeeze(0),
        }

In [ ]:
# Load the labeled dataset
input_texts = queryData['final_TQL'].values # List of input texts
target_queries = queryData['SQL'].values  # List of corresponding target SQL queries

# Split the dataset into train and validation sets
train_input_texts, val_input_texts, train_target_queries, val_target_queries = train_test_split(input_texts, target_queries, test_size=0.2, random_state=42)

In [ ]:
# Create instances of the custom dataset
task_prefix = 'Generate an SQL Query for '
train_dataset = SQLDataset(train_input_texts, train_target_queries, tokenizer, task_prefix)
val_dataset = SQLDataset(val_input_texts, val_target_queries, tokenizer, task_prefix)

In [ ]:
# Define the training hyperparameters
BATCH_SIZE = 8
NUM_EPOCHS = 10
LEARNING_RATE = 2e-5

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Training loop
for epoch in tqdm(range(NUM_EPOCHS)):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluation on validation set
    model.eval()
    total_val_loss = 0
    for batch in val_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)

    print(f'Epoch: {epoch+1}, Validation Loss: {avg_val_loss:.4f}')

# Fine-tuning completed, you can now use the trained model for generating SQL queries
